Nombre: Ulises Gallardo Rodríguez

Tarea #3 Segundo Parcial

In [166]:
import numpy as np
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()


In [167]:
def get_texts_from_file(path_corpus, path_truth):
    tr_txt = []
    tr_y = []
    
    with open(path_corpus, "r") as f_corpus, open(path_truth, "r") as f_truth:
        for tuit in f_corpus:
            tr_txt += [tuit]
            
        for label in f_truth:
            tr_y += [label]
            
    return tr_txt, tr_y

In [168]:
tr_txt, tr_y = get_texts_from_file("./mex_train.txt", "./mex_train_labels.txt")

In [169]:
print(len(tr_txt), " - ", len(tr_y))

5544  -  5544


In [170]:
for i in range(len(tr_txt)):
    tr_txt[i]= tr_txt[i].lower()

## N-grams

In [171]:
def n_grams(tr_txt, indices_palabras, n): #bigramas, fase 1
    indices_palabras["<s>"] = len(indices_palabras)+1
    indices_palabras["</s>"] = len(indices_palabras)+1
    indices_palabras["<UNK>"] = len(indices_palabras)+1
    n+=3 
    Probablilities = np.zeros((n,n), dtype = "int")
    #para el unigrama usar fdist = nltk.FreqDist(corpus_palabras)
    
    for i in range(len(tr_txt)):
        tweet = tr_txt[i]
        tweet = "<s>"+tweet+"</s>" #bigramas
        words = tokenizer.tokenize(tweet)
        #print(words)
        
        for index, word in enumerate(words[1:]):
            #print(words[index], word)
            y = indices_palabras[word]
            x = indices_palabras[words[index]] #prev
            Probablilities[y][x]+=1
    return Probablilities

## Corpus de Prueba

In [172]:
tr_prueba, tr_y_prueba = get_texts_from_file("./prueba.txt", "./prueba.txt")

In [173]:
corpus_palabras = []
for doc in tr_prueba:
    corpus_palabras += tokenizer.tokenize(doc)

In [174]:
corpus_palabras

['I',
 'am',
 'Sam',
 'Sam',
 'I',
 'am',
 'I',
 'do',
 'not',
 'like',
 'green',
 'eggs',
 'and',
 'ham']

In [175]:
indices_palabras = dict()
cnt = 0
V = set(corpus_palabras)
for word in V:
    indices_palabras[word]=cnt
    cnt += 1  

In [176]:
indices_palabras

{'am': 0,
 'like': 1,
 'eggs': 2,
 'I': 3,
 'and': 4,
 'ham': 5,
 'do': 6,
 'green': 7,
 'Sam': 8,
 'not': 9}

In [177]:
Probablilities = n_grams(tr_prueba, indices_palabras, len(V))

In [178]:
Probablilities

array([[0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]])

## Corpus Tweets

In [179]:
corpus_palabras = []
for doc in tr_txt:
    corpus_palabras += tokenizer.tokenize(doc)

In [180]:
len(set(corpus_palabras))

13581

In [181]:
indices_palabras = dict()
cont = 0
V = set(corpus_palabras)
for word in V:
    indices_palabras[word]=cont
    cont += 1    

In [182]:
Probablilities = n_grams(tr_txt, indices_palabras, len(V))

In [183]:
Probablilities

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])